<a href="https://colab.research.google.com/github/RogerioLimaDev/GenerativeAIStudies/blob/main/Imers%C3%A3oAlura_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U google-generativeai

In [4]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)


models/embedding-001
models/text-embedding-004


In [63]:
title='A politicagem nas empresas'
sample_text = ('Título: A politicagem nas empresas \n'
'Artigo completo \n'
'A cada dia que passa, nesses tempos depois da pandemia, vemos a revelação e a expansão de práticas politiqueiras nas corporações'
'a politicagem que antes era uma prática velada, agora se tornou explícita e até incentivada')

embeddings = genai.embed_content(model='models/embedding-001',
                                 content = sample_text,
                                 title = title,
                                 task_type = 'RETRIEVAL_DOCUMENT')
print(embeddings)

{'embedding': [0.018071128, -0.004208832, 0.002418332, 0.029551066, 0.023576984, 0.005678728, -0.03964992, 0.014086303, 0.0056137815, 0.05737507, -0.008916376, -0.02306431, 0.00095636444, -0.021957487, 0.010221862, -0.08030898, 0.022152273, 0.013634455, 0.002301254, -0.05480672, -0.02604267, 0.009565959, -0.014056643, 0.005352377, 0.0114417495, -0.038228337, -0.015356951, -0.04516041, -0.01746807, 0.071001366, -0.037276376, 0.060238168, -0.027671538, -0.029834151, -0.04311001, -0.0047219577, -0.04726804, 0.015689384, 0.028430784, 0.046981733, -0.00055236055, -0.0033050326, -0.022280108, -0.0068897814, -0.009295152, -0.015588593, 0.01135888, 0.034570344, 0.031983398, -0.033248626, -0.022018524, -0.002091092, 0.06009889, -0.021593196, 0.02607847, -0.04494543, 0.06362898, -0.016530408, -0.047383178, -0.025457779, 0.006199433, 0.012536271, 0.04030075, 0.025296483, -0.024265928, -0.04918027, -0.03017365, -0.008761041, 0.049165342, 0.033410214, 0.025579918, -0.04759186, 0.0059350426, -0.0252

In [69]:
document1 = {
    'dispositivo' : 'GearVR',
    'tipo' : 'mobile',
    }

document2 = {
    'dispositivo' : 'Oculus Rift',
    'tipo' : 'PC',
    }

document3 = {
    'dispositivo' : 'Meta Quest2',
    'tipo' : 'mobile',
    }


documents = [document1, document2, document3]
# df = dataframe - como tabela é conhecida no pacote Pandas
df = pd.DataFrame(documents)
df


,dispositivo,tipo
0,GearVR,mobile
1,Oculus Rift,PC
2,Meta Quest2,mobile


In [92]:
model = 'models/embedding-001'

In [93]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content = text,
                                 title = title,
                                 task_type = 'RETRIEVAL_DOCUMENT')['embedding']

In [94]:
df['Embeddings'] = df.apply(lambda row: embed_fn(row['dispositivo'], row['tipo']), axis = 1 )

In [95]:
df

,dispositivo,tipo,Embeddings
0,GearVR,mobile,"[0.041102193, -0.0475803, -0.045406483, -0.035..."
1,Oculus Rift,PC,"[-0.008616337, -0.05483398, -0.045271445, -0.0..."
2,Meta Quest2,mobile,"[0.052182212, -0.050199755, -0.04082415, -0.02..."


In [96]:
# pesquisar consultas escalares
def gerarEmbed_e_consultar(consulta, base, model):
  embeddingConsulta = genai.embed_content(model=model,
                                 content = consulta,
                                 task_type = 'RETRIEVAL_QUERY')

  produtos_escalares = np.dot(np.stack(df['Embeddings']), gerarEmbed_e_consultar(consulta, base, model))
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]['tipo']



In [98]:
consulta = 'Qual é o dispositivo do tipo PC?'
resposta = gerarEmbed_e_consultar(consulta, df, model)
print(resposta.text)

RecursionError: maximum recursion depth exceeded while calling a Python object